# Preface
The purpose of this notebook is to sync the hdf5 data with the merger trees. Currently, halos represented in the hdf5 time steps do not necessarily align with their representations in other time steps (for instance, halo 0 in time step 0 may be different from halo 0 at time step 1. Once finished, this will allow us to add a field (yt_id) to the hdf5 dataset to allow us to track halos through different time steps.

# Preconditon
You must install GraphWiz from https://bobswift.atlassian.net/wiki/spaces/GVIZ/pages/131924165/Graphviz+installation first to be able to produce plots of the trees

In [1]:
import ytree
import h5py
import pandas as pd
from os import walk

# Convert tree data to pandas df
tree_df = pd.DataFrame()

arbor = ytree.load('./trees/tree_0_0_0.dat')
fields = arbor.field_list
print(fields)
trees = list(arbor[:])
for tree in trees:
    for node in tree['tree']:
        node_data = {}
        for key in fields:
            node_data[key] = node[key]
        node_df = pd.DataFrame(node_data, index=[0])
        tree_df = pd.concat([tree_df, node_df])

print(tree_df.head())
print(tree_df.shape)

Additional features and improved performance (usually) by saving this arbor with "save_arbor" and reloading:
	>>> a = ytree.load("./trees/tree_0_0_0.dat")
	>>> fn = a.save_arbor()
	>>> a = ytree.load(fn)
['scale', 'id', 'desc_scale', 'desc_id', 'num_prog', 'pid', 'upid', 'desc_pid', 'phantom', 'sam_Mvir', 'Mvir', 'Rvir', 'rs', 'vrms', 'mmp?', 'scale_of_last_MM', 'vmax', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Jx', 'Jy', 'Jz', 'Spin', 'Breadth_first_ID', 'Depth_first_ID', 'Tree_root_ID', 'Orig_halo_ID', 'Snap_idx', 'Next_coprogenitor_depthfirst_ID', 'Last_progenitor_depthfirst_ID', 'Last_mainleaf_depthfirst_ID', 'Tidal_Force', 'Tidal_ID', 'Rs_Klypin', 'Mvir_all', 'M200b', 'M200c', 'M500c', 'M2500c', 'Xoff', 'Voff', 'Spin_Bullock', 'b_to_a', 'c_to_a', 'A[x]', 'A[y]', 'A[z]', 'b_to_a(500c)', 'c_to_a(500c)', 'A[x](500c)', 'A[y](500c)', 'A[z](500c)', 'T/|U|', 'M_pe_Behroozi', 'M_pe_Diemer', 'Type', 'SM', 'Gas', 'BH_Mass']


Loading tree roots: 100%|██████████████████████████████████████████████| 7934216/7934216 [00:00<00:00, 284626359.10it/s]


     scale     id  desc_scale  desc_id  num_prog  pid  upid  desc_pid  \
0  0.08209  26720     0.00000       -1         6   -1    -1        -1   
0  0.08197  26423     0.08209    26720         1   -1    -1        -1   
0  0.08184  26117     0.08197    26423         2   -1    -1        -1   
0  0.08171  25785     0.08184    26117         1   -1    -1        -1   
0  0.08159  25451     0.08171    25785         3   -1    -1        -1   

   phantom    sam_Mvir  ...  A[x](500c)  A[y](500c)  A[z](500c)   T/|U|  \
0      0.0  53770000.0  ...     0.29397     0.74456    -0.05922  0.8417   
0      0.0  53266700.0  ...     0.28735     0.73221    -0.04249  0.8397   
0      0.0  52763300.0  ...     0.30641     0.79467    -0.02777  0.8370   
0      0.0  52286700.0  ...     0.28301     0.76646    -0.01218  0.8360   
0      0.0  51846700.0  ...     0.26916     0.74800     0.00497  0.8351   

   M_pe_Behroozi  M_pe_Diemer  Type   SM  Gas  BH_Mass  
0     52103000.0   59338000.0   0.0  0.0  0.0      0.

In [3]:
import numpy as np

# Combine halo files into one dataframe
files_df = pd.DataFrame()

data_files = next(walk('./data'), (None, None, []))[2]
for data_file in data_files:
    if "halodat_DD" in data_file:
        path = './data/' + data_file
        file_df = pd.DataFrame()
        loaded_file = h5py.File(path)
        for key in loaded_file:
            cols=[key]
            # Account for datasets with multiple columns
            if (key == 'position'):
                cols=['x', 'y', 'z']
            key_df = pd.DataFrame(loaded_file[key], columns=cols)
            file_df = pd.concat([file_df, key_df], axis=1)
        files_df = pd.concat([files_df, file_df])
            
print(files_df.head())
print(files_df.shape)

   avg_metallicity  gas_fraction  id  num_pop3         x         y         z  \
0     5.983862e-20      0.138768   0         0  0.278435  0.278643  0.251910   
1     6.000776e-20      0.053271   1         0  0.280127  0.282721  0.256123   
2     5.914774e-20      0.124202   2         0  0.279172  0.278518  0.254171   
3     5.871661e-20      0.048896   3         0  0.282602  0.284505  0.261912   
4     5.848676e-20      0.008570   4         0  0.300005  0.332475  0.343459   

     radius  stellar_mass       time     total_mass  
0  0.694017           0.0  68.181023   31758.794922  
1  0.526861           0.0  68.181023   13894.472656  
2  1.365998           0.0  68.181023  242160.812500  
3  1.124173           0.0  68.181023  134974.875000  
4  0.647607           0.0  68.181023   25804.021484  
(19680, 11)


In [4]:
print(files_df.tail())

     avg_metallicity  gas_fraction   id  num_pop3         x         y  \
168     1.751079e-07      0.009565  168         1  0.359949  0.350365   
169     3.383481e-09      0.006097  169         1  0.359590  0.350161   
170     3.608459e-04      0.002041  170         1  0.361918  0.349950   
171     5.686112e-20      0.027060  171         0  0.372365  0.350461   
172     5.820094e-20      0.014492  172         0  0.376950  0.347387   

            z    radius  stellar_mass       time    total_mass  
168  0.347238  3.405976   1410.246094  49.936481  4.042737e+06  
169  0.347430  1.180438   1410.246094  49.936481  1.582190e+05  
170  0.354716  1.986905    687.014160  49.936481  7.648862e+05  
171  0.370550  0.926034      0.000000  49.936481  7.741206e+04  
172  0.379228  0.471002      0.000000  49.936481  9.924623e+03  
